# Программа парсер на Python
## Цели
* Скачать access.log
* Сформировать CSV файл со списком уникальных IP-адресов из get-запросов. Для каждого из них должно быть указано успешных get-запросов с этого адреса, и количество всех остальных get-запросов.
* Вывести на экран список TOP30 IP адресов, с наибольшим количеством успешных get-запросов.

# Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
import numpy as np
import wget
import regex as re

import urllib
from apachelogs import LogParser

Задачу будем решать двумя разными способами, используя как регулярные выражения, так и apachelogs.LogParser.

In [2]:
url = 'http://www.almhuette-raith.at/apache-log/access.log'

Сформируем паттерны для модулей re и apachelogs:

In [3]:
re_compiled = re.compile(r"""
    (?P<ip> ^ (\d{0,3} [\.|\S]) {4} \S+) # ip
    .+
    \" (?P<request> GET|POST|HEAD|OPTIONS) # запрос
    .+
    \D\s (?P<status> \d\d\d) \s # статус
""", re.X)

parser = LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

Спарсим нужные данные.

In [4]:
# Создадим два словаря в которые будем записывать построчно интересующие нас логи.
dict_re = {}
dict_apache = {}
incorrect_logs = 0

with urllib.request.urlopen(url) as fp:
    for i in range(100000):
        # Прочитаем строчку из лог файла.
        log_line = fp.readline().decode("utf-8").strip()

        # На случай если в файле есть пустые строки или не корректные данные зададим блок try / except
        try:
            entry_apache = parser.parse(log_line[:-4])
            dict_apache[i] = [entry_apache.remote_host, entry_apache.request_line.split()[0], entry_apache.final_status]
        except:
            incorrect_logs += 1
            
        entry_re = re_compiled.match(log_line)
        # Добавляем данные в если строка содержит интересующие данные.
        if entry_re:
            dict_re[i] = [entry_re['ip'], entry_re['request'].upper(), int(entry_re['status'])]

print('Количество проблемных логов:', incorrect_logs)

Количество проблемных логов: 1


*В данном случае не получилось спарсить только первую пустую строку.*

Так же файл целиком можно загрузить через wget. А потом прочитать.

    wget.download(url)
    
    with open('access.log') as fp:
        logs = []
        [logs.append(fp.readline().strip()) for _ in range(20000)]

# Формирование CSV файл со списком уникальных IP-адресов из get-запросов

Преобразуем словари в датафреймы.

In [5]:
df_apache = pd.DataFrame.from_dict(dict_apache, orient='index', columns=['ip','request','status']).reset_index(drop=True)
df_re = pd.DataFrame.from_dict(dict_re, orient='index', columns=['ip','request','status']).reset_index(drop=True)

Выполним сверку полученных результатов.

In [6]:
df_apache.equals(df_re)

True

**Данные совпадают. Оба метода работают.**

Оставим только логи содержащие GET запросы.

In [7]:
df_re = df_re.query('request == "GET"')

Определим новый столбец, выделив является ли запрос успешным (2**) или нет.

In [8]:
df_re['status_2**'] = df_re.status.apply(lambda x: 200<=x<300)

In [9]:
df_re.sample(10)

,ip,request,status,status_2**
3570,45.138.145.106,GET,200,True
12208,45.144.0.179,GET,200,True
21045,194.60.76.9,GET,200,True
22999,45.138.6.188,GET,200,True
38020,173.242.124.14,GET,404,False
72723,45.15.143.155,GET,200,True
36614,173.255.176.5,GET,404,False
93493,178.44.47.170,GET,206,True
14461,45.132.51.36,GET,200,True
50258,193.83.238.138,GET,200,True


Сгруппируем данные по ip.

In [10]:
pivot = df_re.pivot_table(index='ip', columns='status_2**', values='request',
                          aggfunc='count') \
             .reset_index()
# Дадим столбцам адекватные имена.
pivot.columns = ['ip','other_req','success_req']

Сохраним .csv

In [11]:
pivot.to_csv('logs_filtered.csv')

# TOP30 IP адресов, с наибольшим количеством успешных get-запросов

In [12]:
print('Уникальных IP:',pivot.shape[0])
pivot.sort_values('success_req', ascending=False).head(30)

Уникальных IP: 2212


,ip,other_req,success_req
604,178.44.47.170,11.0,2771.0
1588,51.210.183.78,NaN,2684.0
1445,45.15.143.155,NaN,1285.0
538,173.255.176.5,2800.0,1234.0
1440,45.144.0.179,NaN,473.0
577,176.222.58.254,NaN,467.0
1419,45.132.207.154,NaN,445.0
1450,45.153.227.55,NaN,444.0
1433,45.138.4.22,NaN,440.0
1449,45.153.227.31,NaN,438.0
